In [4]:
import csv
import urllib
import arcgis
import geopandas as gpd
from arcgis.gis import GIS
import numpy as np
from shapely.geometry import Point
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features.find_locations import find_existing_locations
from arcgis.features import SpatialDataFrame
from arcgis.features import use_proximity
from arcgis.geoenrichment import *
import pandas as pd
from arcgis.features.manage_data import merge_layers
from arcgis.geoenrichment import standard_geography_query, enrich
import seaborn as sns;
sns.set(color_codes=True)
import math
import matplotlib.pyplot as plt
from arcgis.features import summarize_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [5]:
# Xiangchen Zhao
gis = GIS("https://ucsdonline.maps.arcgis.com/home/item.html?id=c74bf4d93f514f0296c1ffd6dda71a73", client_id='S9hCtNsAKhoTpS87')
# Yicen Ma
# gis = GIS("https://ucsdonline.maps.arcgis.com/home/item.html?id=01cee8d17e2446739d57e7714c693624", client_id='yYvN0h2gOFqtCZRA')
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=S9hCtNsAKhoTpS87&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=PEznLigOpbiRtb9yEZMijWH4EtIymg
Enter code obtained on signing in using SAML: ········


/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Successfully logged in as: xiz643_UCSDOnline


In [6]:
# all the feature layer
polyly=gis.content.get('55ae7b83821947618ed692f6de4fe8c8')
pointsly=gis.content.get('1bef20c5bf4c46b897ce6a5788d20221')
centroidly=gis.content.get('c5799774da6c4271b00e8a50011f8da5')

# convert into SEDF
poly=pd.DataFrame.spatial.from_layer(polyly.layers[0])
points=pd.DataFrame.spatial.from_layer(pointsly.layers[0])
centroid=pd.DataFrame.spatial.from_layer(centroidly.layers[0])

# read the updated csv
update1=pd.read_csv('https://raw.githubusercontent.com/opensandiego/sandiego-covid-data/master/sd_daily_city_summary.csv')

In [7]:
# clean updated csv
index=update1.loc[update1['city'].isin (['other ','unknown* ', 'total san diego county residents '])].index
update=update1.drop(index,axis=0).reset_index(drop=True)
update['city']=update['city'].str.lower()
update['city']=update['city'].str.replace(' ', '')


In [8]:
# creat a new dataframe 
newup=pd.DataFrame(update['date'].unique()).T
newup.columns=newup.iloc[0,:]
newup.drop(0,inplace=True)
newup['city']=update['city'].unique()


In [9]:
# implement the new updated dataset with comfirmed cases
for i in range(len(update)):
    #get information from update dataframe
    city=update.loc[i,'city']
    date=update.loc[i,'date']
    value=update.loc[i,'confirmed_cases']
    
    #set value in new table
    index=newup.loc[newup['city']==city].index
    newup.loc[index,date]=value
newup.head()

,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-05,2020-04-07,city
0,11,12,14,13,16,15,18,20,20,24,27,30,32,42,42,carlsbad
1,4,4,6,7,13,17,21,24,24,32,38,52,65,104,120,chulavista
2,2,4,5,5,5,5,6,6,6,6,5,5,5,8,8,delmar
3,2,2,7,8,13,19,NaN,29,29,34,36,38,43,74,85,elcajon
4,4,5,5,7,8,12,14,18,18,19,20,24,25,30,31,encinitas


In [10]:
# implement the polygon layer with undated dataset
newpoly=poly[['FID','COom_adj','SUM_COUNT_','Shape__Length','Shape__Area','SHAPE']]
newpoly=newpoly.rename(columns={'COom_adj':'city','SUM_COUNT_':'zip_included'})
newpoly = newpoly.set_index(['FID'])
newpoly['city']=newpoly['city'].str.lower()
newpoly['city']=newpoly['city'].str.replace(' ', '')

newpoly = pd.merge(newpoly, newup, on='city',how='left')
newpoly.head()

,city,zip_included,Shape__Length,Shape__Area,SHAPE,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-05,2020-04-07
0,bonita,1,34253.928089,3.584046e+07,"{'rings': [[[-13022166.3169882, 3853307.240272...",NaN,NaN,1,1,1,1,2,2,2,4,3,6,6,8,11
1,carlsbad,4,56120.158052,1.444238e+08,"{'rings': [[[-13053082.0181245, 3916226.877610...",11,12,14,13,16,15,18,20,20,24,27,30,32,42,42
2,chulavista,5,84343.620753,2.664249e+08,"{'rings': [[[-13018505.2168385, 3853364.165457...",4,4,6,7,13,17,21,24,24,32,38,52,65,104,120
3,coronado,3,42455.007199,5.108144e+07,"{'rings': [[[-13038884.686413, 3851131.2618482...",NaN,NaN,NaN,0,0,NaN,1,1,1,1,1,2,2,4,4
4,delmar,1,29513.341671,2.436836e+07,"{'rings': [[[-13048633.6831031, 3892747.514411...",2,4,5,5,5,5,6,6,6,6,5,5,5,8,8


In [20]:
# add new column population_at_risk and infection per 10000
risk=pd.read_excel('Selected Conditions by Zip2017_04.07.20.xlsx')
risk.head()

,Year of cases,Enclosing Zip,Outcome,Condition,Total,Age0_14,Age15_24,Age25_44,Age45_64,Age65Plus,TotalRate,Age0_14Rate,Age15_24Rate,Age25_44Rate,Age45_64Rate,Age65PlusRate
0,2017,91901,ED Discharge,Any Mention of Alzheimers Disease and Related ...,133.0,NaN,NaN,NaN,10.0,123.0,738.970997,NaN,NaN,NaN,202.020202,3906.001905
1,2017,91901,ED Discharge,Any Mention of Diabetes,439.0,NaN,27.0,41.0,151.0,220.0,2439.159907,NaN,1056.751468,1018.127638,3050.505051,6986.344871
2,2017,91901,ED Discharge,Any Mention of Lung Cancer,14.0,NaN,NaN,NaN,NaN,13.0,77.786421,NaN,NaN,NaN,NaN,412.829470
3,2017,91901,ED Discharge,Any Mention of Mental Illness,1382.0,44.0,150.0,362.0,427.0,399.0,7678.630959,1326.499849,5870.841487,8989.322076,8626.262626,12670.689108
4,2017,91901,ED Discharge,Any Mention of Obesity,264.0,NaN,8.0,58.0,96.0,102.0,1466.829648,NaN,313.111546,1440.278123,1939.393939,3239.123531


In [32]:
# merge city name with zipcode
city = poly[['COom_adj','SHAPE']]
sd_zipcode = gis.content.get("1fa5dfbe76e5485faa90bdfed3a246c8")
sd_zipcode_sdf = pd.DataFrame.spatial.from_layer(sd_zipcode.layers[0])
zipcode = sd_zipcode_sdf[['ZIP','SHAPE','COMMUNITY']]

c = city['COom_adj'].unique()
z = zipcode['COMMUNITY'].unique()

city = city.rename(columns={'COom_adj':'COMMUNITY'})
city_with_zip = city.merge(zipcode, on='COMMUNITY')
city_with_zip = city_with_zip.drop('SHAPE_y',axis = 1)
city_with_zip = city_with_zip.rename(columns = {'SHAPE_x':'SHAPE'})
city_with_zip=city_with_zip.rename(columns={'COMMUNITY':'city'})
city_with_zip

,city,SHAPE,ZIP
0,Bonita,"{'rings': [[[-13022166.3169882, 3853307.240272...",91902
1,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92008
2,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92009
3,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92010
4,Carlsbad,"{'rings': [[[-13053082.0181245, 3916226.877610...",92011
5,Chula Vista,"{'rings': [[[-13018505.2168385, 3853364.165457...",91911
6,Chula Vista,"{'rings': [[[-13018505.2168385, 3853364.165457...",91910
7,Chula Vista,"{'rings': [[[-13018505.2168385, 3853364.165457...",91915
8,Chula Vista,"{'rings': [[[-13018505.2168385, 3853364.165457...",91913
9,Chula Vista,"{'rings': [[[-13018505.2168385, 3853364.165457...",91914


In [42]:
# new column risk rate per 100,000 population
rate=risk[['Enclosing Zip','TotalRate']].groupby('Enclosing Zip').sum().reset_index()
risk.head()

,Year of cases,Enclosing Zip,Outcome,Condition,Total,Age0_14,Age15_24,Age25_44,Age45_64,Age65Plus,TotalRate,Age0_14Rate,Age15_24Rate,Age25_44Rate,Age45_64Rate,Age65PlusRate
0,2017,91901,ED Discharge,Any Mention of Alzheimers Disease and Related ...,133.0,NaN,NaN,NaN,10.0,123.0,738.970997,NaN,NaN,NaN,202.020202,3906.001905
1,2017,91901,ED Discharge,Any Mention of Diabetes,439.0,NaN,27.0,41.0,151.0,220.0,2439.159907,NaN,1056.751468,1018.127638,3050.505051,6986.344871
2,2017,91901,ED Discharge,Any Mention of Lung Cancer,14.0,NaN,NaN,NaN,NaN,13.0,77.786421,NaN,NaN,NaN,NaN,412.829470
3,2017,91901,ED Discharge,Any Mention of Mental Illness,1382.0,44.0,150.0,362.0,427.0,399.0,7678.630959,1326.499849,5870.841487,8989.322076,8626.262626,12670.689108
4,2017,91901,ED Discharge,Any Mention of Obesity,264.0,NaN,8.0,58.0,96.0,102.0,1466.829648,NaN,313.111546,1440.278123,1939.393939,3239.123531


In [49]:
#merge infection rate by zip code with city name
city_rate=pd.merge(rate,city_with_zip,right_on='ZIP',left_on='Enclosing Zip',how='right')[['city','TotalRate']].groupby('city').sum().reset_index()
city_rate['city']=city_rate['city'].str.lower()
city_rate['city']=city_rate['city'].str.replace(' ', '')
city_rate.head()

,city,TotalRate
0,aguanga,0.000000
1,alpine,42993.665963
2,bonita,44199.408044
3,bonsall,31305.987742
4,borregosprings,11224.698426


In [50]:
pd.merge(newpoly,city_rate,how='left',on='city')

,city,zip_included,Shape__Length,Shape__Area,SHAPE,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,...,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-05,2020-04-07,TotalRate
0,bonita,1,34253.928089,3.584046e+07,"{'rings': [[[-13022166.3169882, 3853307.240272...",NaN,NaN,1,1,1,...,2,2,2,4,3,6,6,8,11,4.419941e+04
1,carlsbad,4,56120.158052,1.444238e+08,"{'rings': [[[-13053082.0181245, 3916226.877610...",11,12,14,13,16,...,18,20,20,24,27,30,32,42,42,1.055344e+05
2,chulavista,5,84343.620753,2.664249e+08,"{'rings': [[[-13018505.2168385, 3853364.165457...",4,4,6,7,13,...,21,24,24,32,38,52,65,104,120,1.639391e+05
3,coronado,3,42455.007199,5.108144e+07,"{'rings': [[[-13038884.686413, 3851131.2618482...",NaN,NaN,NaN,0,0,...,1,1,1,1,1,2,2,4,4,3.200718e+04
4,delmar,1,29513.341671,2.436836e+07,"{'rings': [[[-13048633.6831031, 3892747.514411...",2,4,5,5,5,...,6,6,6,6,5,5,5,8,8,2.490993e+04
5,elcajon,3,90360.312900,2.567031e+08,"{'rings': [[[-13006128.7080638, 3880292.463117...",2,2,7,8,13,...,NaN,29,29,34,36,38,43,74,85,1.708381e+05
6,encinitas,1,50788.925912,6.737541e+07,"{'rings': [[[-13047363.3872105, 3907906.432499...",4,5,5,7,8,...,14,18,18,19,20,24,25,30,31,2.590480e+04
7,escondido,4,194316.478559,6.361946e+08,"{'rings': [[[-13038167.539891, 3939469.4189864...",2,3,5,6,6,...,7,9,9,10,11,14,23,28,32,1.637024e+05
8,fallbrook,1,156136.202021,4.543151e+08,"{'rings': [[[-13078812.156924, 3962537.3270725...",2,2,2,2,2,...,3,4,4,4,4,4,6,6,6,1.810560e+04
9,jamul,1,93950.307831,3.678820e+08,"{'rings': [[[-12999223.9347931, 3866953.795589...",NaN,NaN,NaN,0,1,...,0,NaN,NaN,NaN,NaN,NaN,NaN,2,3,4.233652e+04


In [13]:
#newpoly.spatial.to_featurelayer(title='update poly',gis=gis,tags='dsc198')